## This is a debuging ipython notebook

In [1]:
%load_ext autoreload
import argparse
import warnings
warnings.filterwarnings('ignore')

arg_parser = argparse.ArgumentParser()
        
    # # Add standard arguments
    # if not is_standalone:
    #     # If standalone, set in parser.
arg_parser.add_argument('--output_path', default='.',
                                help='[Default is current directory] Path to '
                                     'output directory.')


    # Arguments to generate the reference
arg_parser.add_argument('-r', '--reference', action='store_true',
                            help='Just generate the reference dataset for mapping')
arg_parser.add_argument('--min_species', type=int, default=30,
                            help='Min number of species in selected orthologous groups. \
                            If not selected it will be estimated such that around 1000 OGs are available.')
arg_parser.add_argument('--dna_reference', default='/Volumes/Untitled/eukaryotes.cdna.fa',
                            help='Reference fasta file that contains nucleotide sequences.')

    # Arguments to map the reads
arg_parser.add_argument('--ref_folder', default=None,
                            help='Folder containing reference files with sequences sorted by species.')
arg_parser.add_argument('--reads', default='/Users/daviddylus/Research/pore2tree/fritz_scripts/pipeline/Nano_reads/ERR1877969.fastq',
                            help='Reads to be mapped to reference.')
# arg_parser.add_argument('--reads', nargs='2',default=None, help='Reads to be mapped to reference.')


# Parse the arguments.
# reference = ['--ref_folder', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test1/db/']
argv = ['--output_path','/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26', 
        '--reads', '/Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_0_0.fq /Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_1_0.fq']
args = arg_parser.parse_args(argv)
print(args)

Namespace(dna_reference='/Volumes/Untitled/eukaryotes.cdna.fa', min_species=30, output_path='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26', reads='/Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_0_0.fq /Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_1_0.fq', ref_folder=None, reference=False)


In [10]:
import glob
output_path='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/'
for files in glob.iglob(output_path+'/**/*consensus.fa', recursive=True):
    print(files)

/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/AMPQE_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/CHICK_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/DROME_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/ACYPI_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/AEDAE_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/ANOCA_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/ATTCE_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/03_mapping_SRR5171076_10X_0/BRAFL_consensus.fa
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5

In [29]:
from Bio import AlignIO
import re

for folder in glob.iglob(output_path+'/05_*', recursive=True):
    print(folder)
    all_coverages = []

    for file in glob.iglob(folder+'/*.phy'):
        align = AlignIO.read(file, "phylip-relaxed")
        for record in align:
            if 'SRR51' in record.id:
                seq = re.sub('-', '', str(record.seq))
                xx = seq.count("X")
                aa = len(seq)-xx
                all_coverages.append((aa/len(seq)))
    print(sum(all_coverages)/len(all_coverages))





/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_10X_0
0.30892088374479326
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_02X_0_0
0.3870602588528758
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_03X_0_0
0.4943459068699639
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_20X_0_0
0.7367354340405972
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_30X_0_0
0.33473355881327294
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_04X_0_0
0.4074008119696601
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_09X_0_0
0.3905378725765558
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/05_align_SRR5171076_1
0.4887434789065503
/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076